In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from src.models.retinaface import cfg_re50
from src.models.retinaface import retina50
import numpy as np
from PIL import Image
import ast
import cv2 as cv
import time
from src.utils.retinaface import PriorBox
from torchvision.ops import nms
from src.utils.arcface import estimate_norm, LANDS_TEMPLATE

In [2]:
def decode(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    """

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes
def decode_landm(pre, priors, variances):
    """Decode landm from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        pre (tensor): landm predictions for loc layers,
            Shape: [num_priors,10]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded landm predictions
    """
    landms = torch.cat((priors[:, :2] + pre[:, :2] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 2:4] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 4:6] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 6:8] * variances[0] * priors[:, 2:],
                        priors[:, :2] + pre[:, 8:10] * variances[0] * priors[:, 2:],
                        ), dim=1)
    return landms
def py_cpu_nms(dets, thresh):
    """Pure Python NMS baseline."""
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep
def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True
def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}
def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = retina50()
model = load_model(model, "checkpoints/RetinaFace-R50.pth", device)
model.eval()
model = model.to(device)

d:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pretrained model from checkpoints/RetinaFace-R50.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456


In [4]:
dummy_input = torch.randn(1, 3, 640, 640).to(device)

torch.onnx.export(
    model,
    dummy_input,
    "checkpoints/Resnet50_Final.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11,
    do_constant_folding=True,
    dynamic_axes={"input": {0: "batch_size"}}
)

In [4]:
image_path = "./testbench/img/testOmid.jpg"
img_raw = cv.imread(image_path, cv.IMREAD_COLOR)

img = np.float32(img_raw)
resize = 1

scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
im_height, im_width, _ = img.shape
img -= (104, 117, 123)
img = img.transpose(2, 0, 1)
img = torch.from_numpy(img).unsqueeze(0)
img = img.to(device)
scale = scale.to(device)

tic = time.time()
loc, conf, landms = model(img)  # forward pass
print('model forward time: {:.4f}'.format(time.time() - tic))

priorbox = PriorBox(cfg_re50, image_size=(im_height, im_width))
priors = priorbox.forward()
priors = priors.to(device)
prior_data = priors.data
boxes = decode(loc.data.squeeze(0), prior_data, cfg_re50['variance'])
boxes = boxes * scale / resize
boxes = boxes.cpu().numpy()
scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
landms = decode_landm(landms.squeeze(0), prior_data, cfg_re50['variance'])
scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                        img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                        img.shape[3], img.shape[2]])
scale1 = scale1.to(device)
landms = landms * scale1 / resize
landms = landms.cpu().detach().numpy()

# ignore low scores
inds = np.where(scores > 0.02)[0]
boxes = boxes[inds]
landms = landms[inds]
scores = scores[inds]

# keep top-K before NMS
order = scores.argsort()[::-1][:5000]
boxes = boxes[order]
landms = landms[order]
scores = scores[order]

# do NMS
dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
keep = py_cpu_nms(dets, 0.4)
# keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
dets = dets[keep, :]
landms = landms[keep]

# keep top-K faster NMS
dets = dets[:11, :]
landms = landms[:11, :]

dets = np.concatenate((dets, landms), axis=1)
i = 0
for d in dets:
    if d[4] < 0.6:
        continue
    text = "{:.4f}".format(d[4])
    d = list(map(int, d))

    lm = landms[i]  # (10,)
    landmarks = np.array([
        [lm[0], lm[1]],   # left eye
        [lm[2], lm[3]],   # right eye
        [lm[4], lm[5]],   # nose
        [lm[6], lm[7]],   # mouth left
        [lm[8], lm[9]],   # mouth right
    ], dtype=np.float32)
    
    image = img_raw.copy()#cv.cvtColor(img_raw, cv.COLOR_BGR2RGB)
    face_lands_norm = cv.estimateAffinePartial2D(landmarks, LANDS_TEMPLATE, method=cv.LMEDS)[0]
    image = cv.warpAffine(img_raw, face_lands_norm, (112, 112), flags=cv.INTER_LINEAR)
    # image = np.transpose(image / 127.5 - 1.0, (2,0,1)).astype(np.float32)
    # cv.imshow("image", image)
    # cv.waitKey(5000)
    # cv.destroyAllWindows()
    name = f"E:/Amir/Projects/face/testbench/img/faces/{i}.jpg"
    cv.imwrite(name, image)
    i+=1

    cv.rectangle(img_raw, (d[0], d[1]), (d[2], d[3]), (0, 0, 255), 2)
    cx = d[0]
    cy = d[1] + 12
    cv.putText(img_raw, text, (cx, cy),
                cv.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

    # landms
    cv.circle(img_raw, (d[5], d[6]), 1, (0, 0, 255), 4)
    cv.circle(img_raw, (d[7], d[8]), 1, (0, 255, 255), 4)
    cv.circle(img_raw, (d[9], d[10]), 1, (255, 0, 255), 4)
    cv.circle(img_raw, (d[11], d[12]), 1, (0, 255, 0), 4)
    cv.circle(img_raw, (d[13], d[14]), 1, (255, 0, 0), 4)

# save image

# name = "./testbench/test.jpg"
# cv.imwrite(name, img_raw)

model forward time: 0.6724


In [6]:
landmarks.shape

(5, 2)